# Face Multiclassification Project
## Thomas More | AI-Frameworks
### Matthias Hefel

In [2]:
from utilities import process_files_in_folder

input_folder = "../data_test/test_images/original_images"  
output_folder = "../data_test/test_images/cleaned_images"
process_files_in_folder(input_folder, output_folder, size=(640, 480))


2025-01-13 23:10:22.425701: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-13 23:10:22.428909: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-13 23:10:22.435358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736806222.446700    1064 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736806222.449844    1064 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 23:10:22.462359: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Crop utility function does not work in this file -> use Test Pipeline in 03_crop_faces_retinaface.ipynb instead

In [1]:

from utilities import process_and_crop_faces

test_image_folder = "../data_test/test_images/cleaned_images"
test_output_folder = "../data_test/faces/test_faces"

process_and_crop_faces(test_image_folder, test_output_folder)

Processing 0511.jpg...


2025-01-13 23:22:43.371556: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [2]:
from utilities import load_facenet_model

label_map_path = "../models/label_map.json"
model_path = "../models/facenet_flow_from_directory2.weights.h5"
model = load_facenet_model(label_map_path, model_path)

2025-01-13 23:36:34.503849: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [12]:
from utilities import predict_faces

test_folder = "../data_test/faces/test_faces"
label_map_path = "../models/label_map.json"

image_folder = "../data_test/test_images/cleaned_images"
output_path = "../submissions/submission.csv"

faces = predict_faces(test_folder, label_map_path, model, output_path, image_folder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Submission saved to ../submissions/submission.csv


In [5]:
faces

['matthias', 'matthias', 'matthias']